### classify newsgroup_20 articles by considering each document a simple mean of its individual word embeddings

**TODO**
 - use weighted averages (tf-id, ppmi, etc)
 - other architectures
 - think about why relu + sigmoid works best

In [1]:
import os
import sys
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Dropout, Input, Flatten, Activation
from keras.layers import Conv1D, MaxPooling1D, Embedding, GlobalMaxPooling1D
from keras.models import Model, Sequential

# my stuff in the helpers/ directory
from helpers import files

Using TensorFlow backend.


In [2]:
np.random.seed(seed=42)

In [3]:
embeddings_index = files.read_glove_100()

In [4]:
texts,labels_index,labels = files.read_20_newgroup_files()  

MAX_NB_WORDS = 20000
MAX_SEQUENCE_LENGTH = 1000
VALIDATION_SPLIT = 0.2
EMBEDDING_DIM=100

tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index

In [5]:
# word_position => embedding for that word
numeric_embeddings_index = {}

embedding_matrix = np.zeros((len(word_index)+1,EMBEDDING_DIM))

for word,i in word_index.items():
    
    if i >= MAX_NB_WORDS:
        continue  
    
    embedding_vector = embeddings_index.get(word)
    
    numeric_embeddings_index[i] = embedding_vector
    
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [6]:
inverse_word_index = {}

for word,i in word_index.items():
    inverse_word_index[i] = word

In [7]:
document_embeddings = []

for i,document_sequence in enumerate(sequences):
    
    individual_word_embeddings = []
    
    for word_position in document_sequence:       
                                  
        word = inverse_word_index[word_position]
        
        word_embedding = embeddings_index.get(word)    

        if word_embedding is None:
            continue
        else:    
            individual_word_embeddings.append(word_embedding)
         
    if len(individual_word_embeddings) == 0:
        # some documents have no text
        # so we assign a zero vector
        individual_word_embeddings = np.zeros(
            EMBEDDING_DIM,dtype='float32').reshape(1,EMBEDDING_DIM)
        
    document_embedding = np.mean(individual_word_embeddings,axis=0)
    
    document_embeddings.append(document_embedding)

In [8]:
data = np.vstack(document_embeddings)
labels = to_categorical(np.asarray(labels)) # mutating
indices = np.arange(data.shape[0])
np.random.shuffle(indices)

data = data[indices]
labels = labels[indices]
num_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

X_train = data[:-num_validation_samples]
y_train = labels[:-num_validation_samples]
X_val = data[-num_validation_samples:]
y_val = labels[-num_validation_samples:]

data.shape,labels.shape

((19997, 100), (19997, 20))

In [9]:
labels

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [12]:
%%time

model = Sequential()
model.add(Dense(512,input_dim=EMBEDDING_DIM))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(512))
model.add(Activation('sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(len(labels_index)))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X_train, y_train,
                    batch_size=32,
                    epochs=10,
                    verbose=1,
                    validation_data=(X_val,y_val))

Train on 15998 samples, validate on 3999 samples
Epoch 1/10
15998/15998 [==============================] - 1s - loss: 2.4783 - acc: 0.1966 - val_loss: 1.9128 - val_acc: 0.3648
Epoch 2/10
15998/15998 [==============================] - 1s - loss: 1.8577 - acc: 0.3674 - val_loss: 1.5983 - val_acc: 0.4586
Epoch 3/10
15998/15998 [==============================] - 1s - loss: 1.6329 - acc: 0.4404 - val_loss: 1.4665 - val_acc: 0.5099
Epoch 4/10
15998/15998 [==============================] - 1s - loss: 1.5127 - acc: 0.4843 - val_loss: 1.4330 - val_acc: 0.5081
Epoch 5/10
15998/15998 [==============================] - 1s - loss: 1.4360 - acc: 0.5131 - val_loss: 1.3323 - val_acc: 0.5491
Epoch 6/10
15998/15998 [==============================] - 1s - loss: 1.3859 - acc: 0.5286 - val_loss: 1.3055 - val_acc: 0.5709
Epoch 7/10
15998/15998 [==============================] - 1s - loss: 1.3440 - acc: 0.5444 - val_loss: 1.2686 - val_acc: 0.5689
Epoch 8/10
15998/15998 [==============================] - 1s -